In [8]:
import torch
from torch.utils.data import TensorDataset, DataLoader

torch.manual_seed(1)

train_X = torch.load("./train_X.pt")
train_Y = torch.load("./train_Y.pt")
valid_X = torch.load("./valid_X.pt")
valid_Y = torch.load("./valid_Y.pt")

train_data = TensorDataset(train_X, train_Y)
valid_data = TensorDataset(valid_X, valid_Y)

In [9]:
from torch import nn

# nn.Moduleを継承してモデルを作成
class Perseptron(nn.Module):
  def __init__(self, input_size, output_size) -> None:
    super().__init__()

    # nn.Linear: 全結合層を作成する
    self.fc = nn.Linear(input_size, output_size, bias=False)

    # 重み行列を正規分布に上書き
    nn.init.normal_(self.fc.weight, 0.0, 1.0)

  def forward(self, x):
    return torch.softmax(self.fc(x), dim=-1)
  
model = Perseptron(train_X.size()[1], 4)
model.load_state_dict(torch.load("./knock71.model.pt"))
loss_fn = nn.CrossEntropyLoss()

In [10]:
from torch import optim

op = optim.SGD(model.parameters(), lr=0.1)

In [11]:
def calc(model, data, label):
  size = len(data)
  pred = model(data)
  loss = loss_fn(pred, label)
  accuracy = (pred.argmax(dim = 1) == label).sum().item() / size

  return loss, accuracy

In [12]:
import time

epochs = 10
batches = [1, 2, 4, 8, 16, 32, 64, 128]
losses = torch.zeros((2, epochs))
accuracies = torch.zeros(2, epochs)

for batch in batches:
  train_dataloader = DataLoader(train_data, batch_size=batch, shuffle=True)
  valid_dataloader = DataLoader(valid_data, batch_size=len(valid_data), shuffle=False)

  start = time.perf_counter()
  for i in range(epochs):
    for j, (data, label) in enumerate(train_dataloader):
      pred = model(data)
      loss = loss_fn(pred, label)

      op.zero_grad()
      loss.backward()
      op.step()
  
  end = time.perf_counter()

  print(f"batch {batch}: {(end - start)/epochs}")

batch 1: 3.3115783255001587
batch 2: 2.162783598199894
batch 4: 1.1647682729002553
batch 8: 0.6625867376002134
batch 16: 0.3550213809001434
batch 32: 0.22646925329972872
batch 64: 0.20119026959946495
batch 128: 0.12629914649951388
